# Part 5: Exploratory Analysis

In [309]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import glob
from ast import literal_eval
from collections import Counter
from tqdm.auto import tqdm
import pyarrow as pa

In [310]:
drg = pd.read_parquet('D://Vignesh/Capstone/combined/drg_parquet/drg.parquet', engine='pyarrow')
cms = pd.read_csv('Hospital_Metrics/Medicare_Inpatient_Hospital_by_Provider_and_Service_2020.csv', encoding='windows-1252')

In [311]:
drg.head()

,billing_type,billing_code,negotiated_rates,ccn
0,MS-DRG,0001,135000.0,050327
1,MS-DRG,0002,135000.0,050327
2,MS-DRG,0003,9000.0,050327
3,MS-DRG,0004,9000.0,050327
4,MS-DRG,0005,135095.0,050327


In [312]:
drg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9777 entries, 0 to 9776
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   billing_type      9777 non-null   object 
 1   billing_code      9777 non-null   object 
 2   negotiated_rates  9777 non-null   float64
 3   ccn               9777 non-null   object 
dtypes: float64(1), object(3)
memory usage: 305.7+ KB


In [313]:
cms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165281 entries, 0 to 165280
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Rndrng_Prvdr_CCN           165281 non-null  int64  
 1   Rndrng_Prvdr_Org_Name      165281 non-null  object 
 2   Rndrng_Prvdr_St            165281 non-null  object 
 3   Rndrng_Prvdr_City          165281 non-null  object 
 4   Rndrng_Prvdr_State_Abrvtn  165281 non-null  object 
 5   Rndrng_Prvdr_State_FIPS    165281 non-null  int64  
 6   Rndrng_Prvdr_Zip5          165281 non-null  int64  
 7   Rndrng_Prvdr_RUCA          165281 non-null  float64
 8   Rndrng_Prvdr_RUCA_Desc     165281 non-null  object 
 9   DRG_Cd                     165281 non-null  int64  
 10  DRG_Desc                   165281 non-null  object 
 11  Tot_Dschrgs                165281 non-null  int64  
 12  Avg_Submtd_Cvrd_Chrg       165281 non-null  float64
 13  Avg_Tot_Pymt_Amt           16

In [314]:
drg = drg.astype({'billing_code': 'str','ccn':'int64'})
drg['billing_code'] = drg.apply(lambda row: row['billing_code'] if row['billing_code'].isdigit() else None, axis=1)

drg.head()

,billing_type,billing_code,negotiated_rates,ccn
0,MS-DRG,0001,135000.0,50327
1,MS-DRG,0002,135000.0,50327
2,MS-DRG,0003,9000.0,50327
3,MS-DRG,0004,9000.0,50327
4,MS-DRG,0005,135095.0,50327


In [315]:
drg.dropna(subset=['billing_code'],inplace=True)
drg = drg.astype({'billing_code': 'int64'})

In [316]:
merged = drg.merge(cms, how='inner', left_on=['ccn','billing_code'], right_on=['Rndrng_Prvdr_CCN','DRG_Cd'])

In [317]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 799 entries, 0 to 798
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   billing_type               799 non-null    object 
 1   billing_code               799 non-null    int64  
 2   negotiated_rates           799 non-null    float64
 3   ccn                        799 non-null    int64  
 4   Rndrng_Prvdr_CCN           799 non-null    int64  
 5   Rndrng_Prvdr_Org_Name      799 non-null    object 
 6   Rndrng_Prvdr_St            799 non-null    object 
 7   Rndrng_Prvdr_City          799 non-null    object 
 8   Rndrng_Prvdr_State_Abrvtn  799 non-null    object 
 9   Rndrng_Prvdr_State_FIPS    799 non-null    int64  
 10  Rndrng_Prvdr_Zip5          799 non-null    int64  
 11  Rndrng_Prvdr_RUCA          799 non-null    float64
 12  Rndrng_Prvdr_RUCA_Desc     799 non-null    object 
 13  DRG_Cd                     799 non-null    int64  

In [318]:
merged.head(10)

,billing_type,billing_code,negotiated_rates,ccn,Rndrng_Prvdr_CCN,Rndrng_Prvdr_Org_Name,Rndrng_Prvdr_St,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,MS-DRG,3,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,3,"""ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRI...",23,1.090380e+06,211395.521740,179499.565220
1,MS-DRG,4,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,4,"TRACH W MV >96 HRS OR PDX EXC FACE, MOUTH & NE...",18,9.469241e+05,153933.111110,138166.722220
2,MS-DRG,5,135095.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,5,LIVER TRANSPLANT WITH MCC OR INTESTINAL TRANSP...,20,1.083387e+06,143484.300000,129179.250000
3,MS-DRG,11,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,11,"TRACHEOSTOMY FOR FACE, MOUTH & NECK DIAGNOSES ...",12,3.949202e+05,68277.750000,59581.916667
4,MS-DRG,23,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,21,3.829408e+05,66471.142857,59667.809524
5,MS-DRG,25,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,25,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,26,3.617819e+05,50147.307692,45107.153846
6,MS-DRG,27,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,27,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,21,1.635656e+05,26104.333333,23466.095238
7,MS-DRG,64,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,64,INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION...,43,1.982089e+05,26364.906977,21787.767442
8,MS-DRG,65,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,65,INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION...,30,8.148413e+04,11613.433333,9173.800000
9,MS-DRG,70,9000.0,50327,50327,Loma Linda University Medical Center,11234 Anderson St,Loma Linda,CA,6,92354,1.0,Metropolitan area core: primary flow within an...,70,NONSPECIFIC CEREBROVASCULAR DISORDERS W MCC,19,1.361019e+05,20190.473684,15949.315789


In [319]:
merged['ccn'].nunique()

15

| Code Type | Code Value | Description | Associated CMS Metric | Metric Details
| --- | --- | --- | --- | --- |
| CPT | 36556 | Under Insertion of Central Venous Access Device | HAI_1 | Central Line Associated Bloodstream Infection
| CPT | 51701 | Under Introduction Procedures on the Bladder | HAI_2 | Catheter Associated Urinary Tract Infections
| CPT | 51702 | Under Introduction Procedures on the Bladder | HAI_2 | Catheter Associated Urinary Tract Infections
| HCPCS | A4314 | Insertion tray with drainage bag with indwelling catheter, Foley type, 2-way latex with coating (Teflon, silicone, silicone elastomer or hydrophilic, etc.) | HAI_2 | Catheter Associated Urinary Tract Infections
| HCPCS | A4315 | Insertion tray with drainage bag with indwelling catheter, Foley type, 2-way, all silicone | HAI_2 | Catheter Associated Urinary Tract Infections
| HCPCS | G9312 | Surgical site infection | HAI_3 | Surgical Site Infection - Colon Surgery
| CPT | 58150 | Under Hysterectomy Procedures | HAI_4 | Surgical Site Infection - Abdominal Hysterectomy
| CPT | 15920 | Under Pressure Ulcers (Decubitus Ulcers) Procedures | PSI-3 | Pressure Ulcer Rate
| CPT | 35800 | Under Repair, Excision, Exploration, Revision Procedures on Arteries and Veins | PSI-9 | Postoperative hemorrhage or hematoma rate
| HCPCS | J1650 | Injection, enoxaparin sodium, 10 mg | PSI-12 | Perioperative pulmonary embolism or deep vein thrombosis rate
| HCPCS | C9604 | Percutaneous transluminal revascularization of or through coronary artery bypass graft (internal mammary, free arterial, venous), any combination of drug-eluting intracoronary stent, atherectomy and angioplasty, including distal protection when performed | MORT_30_CABG | Death rate for CABG surgery patients


PSI 90 is a composite metric of all PSI scores (patient safety scores). Since it should be highly correlated with all surgeries, I think it would be appropriate to add for CCN availible.

In [320]:
code_values = [['CPT','36556'],['CPT','51701'],['CPT','51702'],['HCPCS','A4314'],
        ['HCPCS','A4315'],['HCPCS','G9312'],['CPT','58150'],['CPT','15920'],
        ['CPT','35800'],['HCPCS','J1650'],['HCPCS','C9604']]

cpt = pd.read_parquet('D:\Vignesh\Capstone\combined\cpt_parquet\cpt.parquet')
hcpcs = pd.read_parquet('D:\Vignesh\Capstone\combined\hcpcs_parquet\hcpcs.parquet')

In [321]:
cpt.head()

,billing_type,billing_code,negotiated_rates,ccn
0,CPT,0001A,16.94,290003
1,CPT,0001U,576.00,290003
2,CPT,0002A,28.39,290003
3,CPT,0002M,402.72,290003
4,CPT,0002U,20.00,290003


In [322]:
cpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14771907 entries, 0 to 14771906
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   billing_type      object 
 1   billing_code      object 
 2   negotiated_rates  float64
 3   ccn               object 
dtypes: float64(1), object(3)
memory usage: 450.8+ MB


In [323]:
df = pd.DataFrame(columns=['billing_type','billing_code','negotiated_rates','ccn'])

for [code_type, value] in code_values:
    if code_type == 'CPT':
        temp = cpt[cpt['billing_code'] == value]
        df = pd.concat([df,temp], axis=0, ignore_index=True)
    else:
        temp = hcpcs[hcpcs['billing_code'] == value]
        df = pd.concat([df,temp], axis=0, ignore_index=True)

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12530 entries, 0 to 12529
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   billing_type      12530 non-null  object 
 1   billing_code      12530 non-null  object 
 2   negotiated_rates  12530 non-null  float64
 3   ccn               12530 non-null  object 
dtypes: float64(1), object(3)
memory usage: 391.7+ KB


In [325]:
df.head()

,billing_type,billing_code,negotiated_rates,ccn
0,CPT,36556,221.30,290003
1,CPT,36556,239.31,290039
2,CPT,36556,92.87,290039
3,CPT,36556,309.82,290003
4,CPT,36556,221.30,290007


In [326]:
infections = pd.read_csv('Hospital_Metrics\Healthcare_Associated_Infections-Hospital.csv', 
                         usecols=['Facility ID','Measure ID','Score'])

In [327]:
infections.head()

,Facility ID,Measure ID,Score
0,010001,HAI_1_CILOWER,0.289
1,010001,HAI_1_CIUPPER,1.307
2,010001,HAI_1_DOPC,10024
3,010001,HAI_1_ELIGCASES,10.597
4,010001,HAI_1_NUMERATOR,7


In [328]:
inf_piv = infections.pivot(index='Facility ID',columns='Measure ID', values='Score').reset_index()
inf_piv.head()

Measure ID,Facility ID,HAI_1_CILOWER,HAI_1_CIUPPER,HAI_1_DOPC,HAI_1_ELIGCASES,HAI_1_NUMERATOR,HAI_1_SIR,HAI_2_CILOWER,HAI_2_CIUPPER,HAI_2_DOPC,...,HAI_5_DOPC,HAI_5_ELIGCASES,HAI_5_NUMERATOR,HAI_5_SIR,HAI_6_CILOWER,HAI_6_CIUPPER,HAI_6_DOPC,HAI_6_ELIGCASES,HAI_6_NUMERATOR,HAI_6_SIR
0,010001,0.289,1.307,10024,10.597,7,0.661,0.140,0.570,17731,...,101908,9.412,8,0.850,0.492,0.868,101451,72.686,48,0.660
1,010005,1.791,6.741,3713,2.450,9,3.673,0.487,2.498,8670,...,38413,1.999,0,0.000,0.419,1.575,35686,10.484,9,0.858
2,010006,0.307,1.575,7318,7.924,6,0.757,0.050,0.534,11755,...,62709,4.164,6,1.441,0.015,0.292,54159,22.618,2,0.088
3,010007,Not Available,Not Available,268,0.165,0,Not Available,Not Available,Not Available,1417,...,5484,0.132,0,Not Available,0.023,2.296,5413,2.148,1,0.466
4,010008,Not Available,Not Available,14,0.008,0,Not Available,Not Available,Not Available,488,...,2171,0.051,0,Not Available,Not Available,Not Available,2171,0.398,0,Not Available


In [329]:
col = inf_piv.columns.to_list()
col

['Facility ID',
 'HAI_1_CILOWER',
 'HAI_1_CIUPPER',
 'HAI_1_DOPC',
 'HAI_1_ELIGCASES',
 'HAI_1_NUMERATOR',
 'HAI_1_SIR',
 'HAI_2_CILOWER',
 'HAI_2_CIUPPER',
 'HAI_2_DOPC',
 'HAI_2_ELIGCASES',
 'HAI_2_NUMERATOR',
 'HAI_2_SIR',
 'HAI_3_CILOWER',
 'HAI_3_CIUPPER',
 'HAI_3_DOPC',
 'HAI_3_ELIGCASES',
 'HAI_3_NUMERATOR',
 'HAI_3_SIR',
 'HAI_4_CILOWER',
 'HAI_4_CIUPPER',
 'HAI_4_DOPC',
 'HAI_4_ELIGCASES',
 'HAI_4_NUMERATOR',
 'HAI_4_SIR',
 'HAI_5_CILOWER',
 'HAI_5_CIUPPER',
 'HAI_5_DOPC',
 'HAI_5_ELIGCASES',
 'HAI_5_NUMERATOR',
 'HAI_5_SIR',
 'HAI_6_CILOWER',
 'HAI_6_CIUPPER',
 'HAI_6_DOPC',
 'HAI_6_ELIGCASES',
 'HAI_6_NUMERATOR',
 'HAI_6_SIR']

In [330]:
inf_piv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4848 entries, 0 to 4847
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Facility ID      4848 non-null   object
 1   HAI_1_CILOWER    4848 non-null   object
 2   HAI_1_CIUPPER    4848 non-null   object
 3   HAI_1_DOPC       4848 non-null   object
 4   HAI_1_ELIGCASES  4848 non-null   object
 5   HAI_1_NUMERATOR  4848 non-null   object
 6   HAI_1_SIR        4848 non-null   object
 7   HAI_2_CILOWER    4848 non-null   object
 8   HAI_2_CIUPPER    4848 non-null   object
 9   HAI_2_DOPC       4848 non-null   object
 10  HAI_2_ELIGCASES  4848 non-null   object
 11  HAI_2_NUMERATOR  4848 non-null   object
 12  HAI_2_SIR        4848 non-null   object
 13  HAI_3_CILOWER    4848 non-null   object
 14  HAI_3_CIUPPER    4848 non-null   object
 15  HAI_3_DOPC       4848 non-null   object
 16  HAI_3_ELIGCASES  4848 non-null   object
 17  HAI_3_NUMERATOR  4848 non-null   

In [331]:
safety = pd.read_csv('Hospital_Metrics\CMS_PSI_6_decimal_file.csv', 
                         usecols=['Facility ID','Measure ID','Rate'])

In [332]:
safety.head()

,Facility ID,Measure ID,Rate
0,010001,PSI_03,0.231881
1,010001,PSI_06,0.167117
2,010001,PSI_08,0.095461
3,010001,PSI_09,2.330697
4,010001,PSI_10,0.609540


In [333]:
saf_piv = safety.pivot(index='Facility ID',columns='Measure ID',values='Rate').reset_index()
saf_piv.head()

Measure ID,Facility ID,PSI_03,PSI_06,PSI_08,PSI_09,PSI_10,PSI_11,PSI_12,PSI_13,PSI_14,PSI_15,PSI_90
0,010001,0.231881,0.167117,0.095461,2.330697,0.609540,8.923957,3.328297,5.980974,0.649012,1.209750,1.005236
1,010005,0.859076,0.173883,0.057663,2.075276,0.756613,6.869380,2.542284,3.440264,0.762277,0.866989,0.908322
2,010006,1.829317,0.261801,0.046622,3.457041,0.651895,3.893786,2.798522,3.721339,0.675144,1.334372,1.099465
3,010007,0.319664,0.183425,0.072035,2.366388,0.914303,6.005962,3.799432,4.048369,Not Available,1.018783,0.993102
4,010008,0.498143,0.185437,0.073861,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available


In [334]:
measure = inf_piv.merge(saf_piv, how='inner', on='Facility ID')
measure.head()

Measure ID,Facility ID,HAI_1_CILOWER,HAI_1_CIUPPER,HAI_1_DOPC,HAI_1_ELIGCASES,HAI_1_NUMERATOR,HAI_1_SIR,HAI_2_CILOWER,HAI_2_CIUPPER,HAI_2_DOPC,...,PSI_06,PSI_08,PSI_09,PSI_10,PSI_11,PSI_12,PSI_13,PSI_14,PSI_15,PSI_90
0,010001,0.289,1.307,10024,10.597,7,0.661,0.140,0.570,17731,...,0.167117,0.095461,2.330697,0.609540,8.923957,3.328297,5.980974,0.649012,1.209750,1.005236
1,010005,1.791,6.741,3713,2.450,9,3.673,0.487,2.498,8670,...,0.173883,0.057663,2.075276,0.756613,6.869380,2.542284,3.440264,0.762277,0.866989,0.908322
2,010006,0.307,1.575,7318,7.924,6,0.757,0.050,0.534,11755,...,0.261801,0.046622,3.457041,0.651895,3.893786,2.798522,3.721339,0.675144,1.334372,1.099465
3,010007,Not Available,Not Available,268,0.165,0,Not Available,Not Available,Not Available,1417,...,0.183425,0.072035,2.366388,0.914303,6.005962,3.799432,4.048369,Not Available,1.018783,0.993102
4,010008,Not Available,Not Available,14,0.008,0,Not Available,Not Available,Not Available,488,...,0.185437,0.073861,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available


In [335]:
measure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4813 entries, 0 to 4812
Data columns (total 48 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Facility ID      4813 non-null   object
 1   HAI_1_CILOWER    4813 non-null   object
 2   HAI_1_CIUPPER    4813 non-null   object
 3   HAI_1_DOPC       4813 non-null   object
 4   HAI_1_ELIGCASES  4813 non-null   object
 5   HAI_1_NUMERATOR  4813 non-null   object
 6   HAI_1_SIR        4813 non-null   object
 7   HAI_2_CILOWER    4813 non-null   object
 8   HAI_2_CIUPPER    4813 non-null   object
 9   HAI_2_DOPC       4813 non-null   object
 10  HAI_2_ELIGCASES  4813 non-null   object
 11  HAI_2_NUMERATOR  4813 non-null   object
 12  HAI_2_SIR        4813 non-null   object
 13  HAI_3_CILOWER    4813 non-null   object
 14  HAI_3_CIUPPER    4813 non-null   object
 15  HAI_3_DOPC       4813 non-null   object
 16  HAI_3_ELIGCASES  4813 non-null   object
 17  HAI_3_NUMERATOR  4813 non-null   

In [336]:
measure.dtypes

Measure ID
Facility ID        object
HAI_1_CILOWER      object
HAI_1_CIUPPER      object
HAI_1_DOPC         object
HAI_1_ELIGCASES    object
HAI_1_NUMERATOR    object
HAI_1_SIR          object
HAI_2_CILOWER      object
HAI_2_CIUPPER      object
HAI_2_DOPC         object
HAI_2_ELIGCASES    object
HAI_2_NUMERATOR    object
HAI_2_SIR          object
HAI_3_CILOWER      object
HAI_3_CIUPPER      object
HAI_3_DOPC         object
HAI_3_ELIGCASES    object
HAI_3_NUMERATOR    object
HAI_3_SIR          object
HAI_4_CILOWER      object
HAI_4_CIUPPER      object
HAI_4_DOPC         object
HAI_4_ELIGCASES    object
HAI_4_NUMERATOR    object
HAI_4_SIR          object
HAI_5_CILOWER      object
HAI_5_CIUPPER      object
HAI_5_DOPC         object
HAI_5_ELIGCASES    object
HAI_5_NUMERATOR    object
HAI_5_SIR          object
HAI_6_CILOWER      object
HAI_6_CIUPPER      object
HAI_6_DOPC         object
HAI_6_ELIGCASES    object
HAI_6_NUMERATOR    object
HAI_6_SIR          object
PSI_03             object
P

In [337]:
col = measure.columns.to_list()
col.remove('Facility ID')
for c in col:
    measure[c] = pd.to_numeric(measure[c],errors='coerce')
measure['Facility ID'] = pd.to_numeric(measure['Facility ID'],errors='coerce')
measure.dropna(subset=['Facility ID'],inplace=True)
measure.drop_duplicates(subset=['Facility ID'],inplace=True)
measure = measure.astype({'Facility ID':'int64'})
types = {c:'float64' for c in col}
measure = measure.astype(types)


In [338]:
cost = pd.read_csv('Hospital_Metrics\Hospital_Cost_Report_2019.csv', 
                         usecols=['Provider CCN','City', 'State Code', 'Zip Code',
                                'Rural Versus Urban','CCN Facility Type',
                                'Fiscal Year Begin Date', 'Fiscal Year End Date',
                                'Number of Interns and Residents (FTE)', 'Number of Beds',
                                'Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders',
                                'Total Discharges (V + XVIII + XIX + Unknown)',
                                'Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds',
                                'Hospital Number of Beds For Adults &amp; Peds',
                                'Hospital Total Bed Days Available For Adults &amp; Peds',
                                'Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds'
                                ])
cost.head()

,Provider CCN,City,State Code,Zip Code,Rural Versus Urban,CCN Facility Type,Fiscal Year Begin Date,Fiscal Year End Date,Number of Interns and Residents (FTE),Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders,Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Hospital Number of Beds For Adults &amp; Peds,Hospital Total Bed Days Available For Adults &amp; Peds,Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds
0,40018,VAN BUREN,AR,72956,U,STH,11/01/2018,12/31/2018,NaN,39.0,168.0,481.0,444.0,31.0,1891.0,168.0
1,100044,STUART,FL,34995,U,STH,10/01/2018,12/31/2018,NaN,521.0,26952.0,31535.0,26412.0,447.0,41124.0,26952.0
2,450090,GAINSVILLE,TX,76240-,R,STH,10/01/2018,11/30/2018,NaN,36.0,175.0,5205.0,534.0,31.0,1708.0,175.0
3,40055,FORT SMITH,AR,72901,U,STH,11/01/2018,12/31/2018,4.26,320.0,2077.0,10556.0,9015.0,289.0,17629.0,2077.0
4,201302,BOOTHBAY HARBOR,ME,04538-,R,CAH,10/01/2018,12/31/2018,NaN,25.0,382.0,19312.0,1377.0,21.0,7665.0,382.0


In [339]:
cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6118 entries, 0 to 6117
Data columns (total 16 columns):
 #   Column                                                                       Non-Null Count  Dtype  
---  ------                                                                       --------------  -----  
 0   Provider CCN                                                                 6118 non-null   int64  
 1   City                                                                         6118 non-null   object 
 2   State Code                                                                   6118 non-null   object 
 3   Zip Code                                                                     6118 non-null   object 
 4   Rural Versus Urban                                                           6049 non-null   object 
 5   CCN Facility Type                                                            6118 non-null   object 
 6   Fiscal Year Begin Date                  

In [340]:
cost[['Fiscal Year Begin Date', 'Fiscal Year End Date']]= cost[['Fiscal Year Begin Date', 'Fiscal Year End Date']].apply(pd.to_datetime)
cost['Zip Code'] = cost['Zip Code'].str[:5]
cat = ['City','State Code','Zip Code', 'Rural Versus Urban', 'CCN Facility Type']
types = {c:'category' for c in cat}
cost = cost.astype(types)
cost.head()

,Provider CCN,City,State Code,Zip Code,Rural Versus Urban,CCN Facility Type,Fiscal Year Begin Date,Fiscal Year End Date,Number of Interns and Residents (FTE),Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders,Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Hospital Number of Beds For Adults &amp; Peds,Hospital Total Bed Days Available For Adults &amp; Peds,Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds
0,40018,VAN BUREN,AR,72956,U,STH,2018-11-01,2018-12-31,NaN,39.0,168.0,481.0,444.0,31.0,1891.0,168.0
1,100044,STUART,FL,34995,U,STH,2018-10-01,2018-12-31,NaN,521.0,26952.0,31535.0,26412.0,447.0,41124.0,26952.0
2,450090,GAINSVILLE,TX,76240,R,STH,2018-10-01,2018-11-30,NaN,36.0,175.0,5205.0,534.0,31.0,1708.0,175.0
3,40055,FORT SMITH,AR,72901,U,STH,2018-11-01,2018-12-31,4.26,320.0,2077.0,10556.0,9015.0,289.0,17629.0,2077.0
4,201302,BOOTHBAY HARBOR,ME,04538,R,CAH,2018-10-01,2018-12-31,NaN,25.0,382.0,19312.0,1377.0,21.0,7665.0,382.0


In [341]:
cost['Time Period'] = cost['Fiscal Year End Date'] - cost['Fiscal Year Begin Date']
cost.head()

,Provider CCN,City,State Code,Zip Code,Rural Versus Urban,CCN Facility Type,Fiscal Year Begin Date,Fiscal Year End Date,Number of Interns and Residents (FTE),Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders,Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Hospital Number of Beds For Adults &amp; Peds,Hospital Total Bed Days Available For Adults &amp; Peds,Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Time Period
0,40018,VAN BUREN,AR,72956,U,STH,2018-11-01,2018-12-31,NaN,39.0,168.0,481.0,444.0,31.0,1891.0,168.0,60 days
1,100044,STUART,FL,34995,U,STH,2018-10-01,2018-12-31,NaN,521.0,26952.0,31535.0,26412.0,447.0,41124.0,26952.0,91 days
2,450090,GAINSVILLE,TX,76240,R,STH,2018-10-01,2018-11-30,NaN,36.0,175.0,5205.0,534.0,31.0,1708.0,175.0,60 days
3,40055,FORT SMITH,AR,72901,U,STH,2018-11-01,2018-12-31,4.26,320.0,2077.0,10556.0,9015.0,289.0,17629.0,2077.0,60 days
4,201302,BOOTHBAY HARBOR,ME,04538,R,CAH,2018-10-01,2018-12-31,NaN,25.0,382.0,19312.0,1377.0,21.0,7665.0,382.0,91 days


In [342]:
columns_to_convert = ['Total Discharges (V + XVIII + XIX + Unknown)', 'Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders',
                      'Hospital Total Bed Days Available For Adults &amp; Peds',
                      'Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds',
                      'Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds']

for column in columns_to_convert:
    cost[column] = cost[column] * (pd.Timedelta('365 days') / cost['Time Period'])

In [343]:
cost.head()

,Provider CCN,City,State Code,Zip Code,Rural Versus Urban,CCN Facility Type,Fiscal Year Begin Date,Fiscal Year End Date,Number of Interns and Residents (FTE),Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders,Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Hospital Number of Beds For Adults &amp; Peds,Hospital Total Bed Days Available For Adults &amp; Peds,Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Time Period
0,40018,VAN BUREN,AR,72956,U,STH,2018-11-01,2018-12-31,NaN,39.0,1022.000000,2926.083333,2701.000000,31.0,11503.583333,1022.000000,60 days
1,100044,STUART,FL,34995,U,STH,2018-10-01,2018-12-31,NaN,521.0,108104.175824,126486.538462,105938.241758,447.0,164947.912088,108104.175824,91 days
2,450090,GAINSVILLE,TX,76240,R,STH,2018-10-01,2018-11-30,NaN,36.0,1064.583333,31663.750000,3248.500000,31.0,10390.333333,1064.583333,60 days
3,40055,FORT SMITH,AR,72901,U,STH,2018-11-01,2018-12-31,4.26,320.0,12635.083333,64215.666667,54841.250000,289.0,107243.083333,12635.083333,60 days
4,201302,BOOTHBAY HARBOR,ME,04538,R,CAH,2018-10-01,2018-12-31,NaN,25.0,1532.197802,77460.219780,5523.131868,21.0,30744.230769,1532.197802,91 days


In [344]:
cost_measure = measure.merge(cost,how='inner',left_on='Facility ID',right_on='Provider CCN')

In [345]:
cost_measure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4668 entries, 0 to 4667
Data columns (total 65 columns):
 #   Column                                                                       Non-Null Count  Dtype          
---  ------                                                                       --------------  -----          
 0   Facility ID                                                                  4668 non-null   int64          
 1   HAI_1_CILOWER                                                                1896 non-null   float64        
 2   HAI_1_CIUPPER                                                                2098 non-null   float64        
 3   HAI_1_DOPC                                                                   3736 non-null   float64        
 4   HAI_1_ELIGCASES                                                              3736 non-null   float64        
 5   HAI_1_NUMERATOR                                                              3736 non-null

In [346]:
cost_measure['Facility ID'].nunique() 

4595

In [347]:
df = df.astype({'ccn':'int64'})
final = df.merge(right=cost_measure,how='inner',left_on='ccn',right_on='Facility ID')
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12266 entries, 0 to 12265
Data columns (total 69 columns):
 #   Column                                                                       Non-Null Count  Dtype          
---  ------                                                                       --------------  -----          
 0   billing_type                                                                 12266 non-null  object         
 1   billing_code                                                                 12266 non-null  object         
 2   negotiated_rates                                                             12266 non-null  float64        
 3   ccn                                                                          12266 non-null  int64          
 4   Facility ID                                                                  12266 non-null  int64          
 5   HAI_1_CILOWER                                                                9280 non-nu

In [348]:
final.describe()

,negotiated_rates,ccn,Facility ID,HAI_1_CILOWER,HAI_1_CIUPPER,HAI_1_DOPC,HAI_1_ELIGCASES,HAI_1_NUMERATOR,HAI_1_SIR,HAI_2_CILOWER,...,Provider CCN,Number of Interns and Residents (FTE),Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders,Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Hospital Number of Beds For Adults &amp; Peds,Hospital Total Bed Days Available For Adults &amp; Peds,Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Time Period
count,12266.000000,12266.000000,12266.000000,9280.000000,9842.000000,12035.000000,12035.000000,12035.000000,9842.000000,9745.000000,...,12266.000000,6064.000000,12266.000000,12266.000000,12266.000000,12254.000000,12254.000000,12254.000000,12266.000000,12266
mean,3244.517782,185144.097913,185144.097913,0.521524,1.993169,12107.750478,12.556499,12.206647,0.977725,0.401496,...,185144.097913,123.892549,336.744415,17740.207698,102317.564451,66250.351717,272.228252,98853.110898,17740.207698,361 days 01:21:14.351866948
std,48042.677932,149006.174367,149006.174367,0.445988,1.167791,15704.893894,17.370191,19.722546,0.687075,0.331791,...,149006.174367,180.711458,405.220490,21808.537927,120720.762499,85128.275756,327.849085,119622.999221,21808.537927,29 days 05:10:05.954698610
min,0.490000,10001.000000,10001.000000,0.004000,0.403000,4.000000,0.001000,0.000000,0.000000,0.003000,...,10001.000000,0.080000,8.000000,73.000000,449.000000,203.557692,8.000000,2928.000000,73.000000,61 days 00:00:00
25%,34.440000,60075.000000,60075.000000,0.214000,1.271000,2705.000000,2.152000,1.000000,0.548000,0.155000,...,60075.000000,16.280000,109.000000,5055.851648,32414.807692,16079.000000,94.000000,32574.244505,5055.851648,364 days 00:00:00
50%,118.635000,100157.000000,100157.000000,0.420000,1.633000,6763.000000,6.356000,5.000000,0.851000,0.315000,...,100157.000000,34.480000,224.000000,12257.000000,64402.445055,42248.750000,192.000000,69500.412088,12257.000000,364 days 00:00:00
75%,737.200000,290041.000000,290041.000000,0.768000,2.378000,14447.000000,15.166000,14.000000,1.257000,0.596000,...,290041.000000,166.050000,396.000000,22766.000000,118028.000000,82277.417582,320.000000,117972.786885,22766.000000,365 days 00:00:00
max,999999.990000,670076.000000,670076.000000,4.081000,11.146000,89310.000000,95.144000,118.000000,5.869000,1.794000,...,670076.000000,1233.830000,2735.000000,131197.445055,815529.326923,538474.271978,2245.000000,821676.167582,131197.445055,365 days 00:00:00


In [349]:
final.drop(columns=['Facility ID', 'Provider CCN', 'Time Period'], inplace=True)
final['Procedure'] = final['billing_type'] + ' ' + final['billing_code']


In [350]:
col = final.columns.to_list()
col

['billing_type',
 'billing_code',
 'negotiated_rates',
 'ccn',
 'HAI_1_CILOWER',
 'HAI_1_CIUPPER',
 'HAI_1_DOPC',
 'HAI_1_ELIGCASES',
 'HAI_1_NUMERATOR',
 'HAI_1_SIR',
 'HAI_2_CILOWER',
 'HAI_2_CIUPPER',
 'HAI_2_DOPC',
 'HAI_2_ELIGCASES',
 'HAI_2_NUMERATOR',
 'HAI_2_SIR',
 'HAI_3_CILOWER',
 'HAI_3_CIUPPER',
 'HAI_3_DOPC',
 'HAI_3_ELIGCASES',
 'HAI_3_NUMERATOR',
 'HAI_3_SIR',
 'HAI_4_CILOWER',
 'HAI_4_CIUPPER',
 'HAI_4_DOPC',
 'HAI_4_ELIGCASES',
 'HAI_4_NUMERATOR',
 'HAI_4_SIR',
 'HAI_5_CILOWER',
 'HAI_5_CIUPPER',
 'HAI_5_DOPC',
 'HAI_5_ELIGCASES',
 'HAI_5_NUMERATOR',
 'HAI_5_SIR',
 'HAI_6_CILOWER',
 'HAI_6_CIUPPER',
 'HAI_6_DOPC',
 'HAI_6_ELIGCASES',
 'HAI_6_NUMERATOR',
 'HAI_6_SIR',
 'PSI_03',
 'PSI_06',
 'PSI_08',
 'PSI_09',
 'PSI_10',
 'PSI_11',
 'PSI_12',
 'PSI_13',
 'PSI_14',
 'PSI_15',
 'PSI_90',
 'City',
 'State Code',
 'Zip Code',
 'Rural Versus Urban',
 'CCN Facility Type',
 'Fiscal Year Begin Date',
 'Fiscal Year End Date',
 'Number of Interns and Residents (FTE)',
 'Number o

In [351]:
col_work = ['ccn', 
            'Procedure',
            'State Code',
            'Rural Versus Urban',
            'PSI_03',
            'PSI_06',
            'PSI_08',
            'PSI_09',
            'PSI_10',
            'PSI_11',
            'PSI_12',
            'PSI_13',
            'PSI_14',
            'PSI_15',
            'PSI_90',
            'HAI_1_SIR',
            'HAI_2_SIR',
            'HAI_3_SIR',
            'HAI_4_SIR',
            'HAI_5_SIR',
            'HAI_6_SIR',
            'Number of Interns and Residents (FTE)',
            'Number of Beds',
            'Total Discharges (V + XVIII + XIX + Unknown)',
            'Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders',
            'Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds',
            'Hospital Number of Beds For Adults &amp; Peds',
            'Hospital Total Bed Days Available For Adults &amp; Peds',
            'Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds',
            'negotiated_rates']

df = final[col_work]
df.head()

,ccn,Procedure,State Code,Rural Versus Urban,PSI_03,PSI_06,PSI_08,PSI_09,PSI_10,PSI_11,...,HAI_6_SIR,Number of Interns and Residents (FTE),Number of Beds,Total Discharges (V + XVIII + XIX + Unknown),Total Days (V + XVIII + XIX + Unknown) + Total for all Subproviders,Hospital Total Days (V + XVIII + XIX + Unknown) For Adults &amp; Peds,Hospital Number of Beds For Adults &amp; Peds,Hospital Total Bed Days Available For Adults &amp; Peds,Hospital Total Discharges (V + XVIII + XIX + Unknown) For Adults &amp; Peds,negotiated_rates
0,290003,CPT 36556,NV,U,0.13266,0.184709,0.114236,1.56662,0.650623,9.818181,...,0.109,31.44,604.0,32823.928571,207129.478022,126462.472527,424.0,140216.153846,32823.928571,221.30
1,290003,CPT 36556,NV,U,0.13266,0.184709,0.114236,1.56662,0.650623,9.818181,...,0.109,31.44,604.0,32823.928571,207129.478022,126462.472527,424.0,140216.153846,32823.928571,309.82
2,290003,CPT 36556,NV,U,0.13266,0.184709,0.114236,1.56662,0.650623,9.818181,...,0.109,31.44,604.0,32823.928571,207129.478022,126462.472527,424.0,140216.153846,32823.928571,273.50
3,290003,CPT 36556,NV,U,0.13266,0.184709,0.114236,1.56662,0.650623,9.818181,...,0.109,31.44,604.0,32823.928571,207129.478022,126462.472527,424.0,140216.153846,32823.928571,106.14
4,290003,CPT 36556,NV,U,0.13266,0.184709,0.114236,1.56662,0.650623,9.818181,...,0.109,31.44,604.0,32823.928571,207129.478022,126462.472527,424.0,140216.153846,32823.928571,171.56


In [355]:
df.info()
df = df.astype({'ccn':'category', 'Procedure':'category'})

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12266 entries, 0 to 12265
Data columns (total 30 columns):
 #   Column                                                                       Non-Null Count  Dtype   
---  ------                                                                       --------------  -----   
 0   ccn                                                                          12266 non-null  int64   
 1   Procedure                                                                    12266 non-null  object  
 2   State Code                                                                   12266 non-null  category
 3   Rural Versus Urban                                                           12266 non-null  category
 4   PSI_03                                                                       11476 non-null  float64 
 5   PSI_06                                                                       11476 non-null  float64 
 6   PSI_08                        

In [357]:
procedure = df.groupby(['Procedure'])
sns.boxplot